<a href="https://colab.research.google.com/github/BeommoKoo-dev/recommender-system/blob/beommo/book_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keybert
!pip install POT

In [3]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
import gensim
from gensim.models import KeyedVectors
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

from gensim.models import LdaModel
from gensim import corpora
from sklearn.manifold import TSNE
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import pandas as pd
import json
import csv
import spacy

In [ ]:
data = []
book_ids = []
book_summaries = []
synopsis = []
bookCount = 200
keywordCount = 15
kw_model = KeyBERT()
sp_model = spacy.load("en_core_web_sm")

with open('/content/sample_data/booksummaries.txt', 'r') as f:
    bookReader = csv.reader(f, dialect='excel-tab')
    count = 0
    for bookData in bookReader:
      if count >= bookCount :
        break
      if count % 10 == 0:
        print(f'Reading File.. ({count}/{bookCount})')

      data.append(bookData)
      count += 1

# convert data to pandas dataframe
books = pd.DataFrame.from_records(data, columns=['book_id', 'freebase_id', 'book_title', 'author', 'publication_date', 'genre', 'summary'])

In [ ]:
book_ids = list(books['book_id'])
bookSummary = list(books['summary'])

In [ ]:
count = 0

for book_idx in range(0, bookCount):
    if count >= bookCount:
        break
    if count % 10 == 0:
        print(f'Reading File.. ({count}/{bookCount})')

    keywords = kw_model.extract_keywords(bookSummary[book_idx], top_n=keywordCount, vectorizer=KeyphraseCountVectorizer())
    keyList = [keyword[0] for keyword in keywords]
    # extracting needless names
    filtered_keywords = []
    for keyword in keyList:
        doc = sp_model(keyword)
        is_name = any(token.ent_type_ == "PERSON" and token.ent_iob_ != '0' for token in doc)

        if not is_name:
            filtered_keywords.append(keyword)

    keyString = ', '.join(filtered_keywords)

    synopsis.append(keyString)

    count += 1

In [ ]:
# Writing .csv
csvFilePath = "book_keyword_extraction_" + str(keywordCount) + ".csv"
with open(csvFilePath, 'w', newline='', encoding='utf-8') as csvFile:
    headers = ['BookId', "Keywords"]
    writer = csv.DictWriter(csvFile, fieldnames=headers)
    writer.writeheader()

    for idx in range(0, bookCount):
        writer.writerow({'BookId': book_ids[idx], 'Keywords': synopsis[idx]})

In [ ]:
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
import csv
book_data = open("/content/sample_data/book_keyword_extraction_15.csv", "r")
movie_data = open("/content/sample_data/movie_keyword_extraction_15.csv", "r")

In [7]:
book_rdr = csv.reader(book_data)
movie_rdr = csv.reader(movie_data)

In [8]:
books = {book[0]:book[1] for book in book_rdr}
movies = {movie[0]:movie[1] for movie in movie_rdr}

# for book in book_rdr :
#   books.append(book)

# for movie in movie_rdr :
#   movies.append(movie)

In [ ]:
distance = model.wmdistance(books['620'], movies['tt0105112'])

In [ ]:
print(movies)

In [35]:
def recommend_movie(book_id, recommend_count) :
  if book_id not in books :
    print("there is no book")
    return
  book_keyword = books[book_id]
  if len(book_keyword) == 0 :
    print("there is no book summarization")
    return
  count = 0
  res = []

  for movie_id, movie_keyword in movies.items() :
    if len(movie_keyword) == 0 :
      continue
    distance = model.wmdistance(book_keyword, movie_keyword)
    res.append({'movie_id' : movie_id, 'distance' : distance})

  sorted_res = sorted(res, key=lambda x: x['distance'])
  # print(sorted_res)
  return sorted_res[:recommend_count]


In [ ]:
res = recommend_movie('620', 2)
print(res)